In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import csv
from sklearn.metrics import brier_score_loss
import pickle
from numpy import std, mean, sqrt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# read csv files into dataframes
workers = pd.read_csv('datasets/participants.csv')
results = pd.read_csv('datasets/results.csv')
sample = pd.read_csv('datasets/defendants.csv')

## Evaluation
Participants should be able to properly assess (1) their own performance, (2) the risk assessment model's performance (accuracy and fairnes):

In [3]:
cols = ['participant_degree', 'participant_age']
for col in cols:
    col_zscore = col + '_zscore'
    workers[col_zscore] = (workers[col] - workers[col].mean())/workers[col].std(ddof=0)

In [4]:
# calculate each participants' rank amongst all participants as a proxy for relative performance
workers['rank'] = workers['participant_brier_score'].rank(method='dense', ascending=False)

### 1. Self-reported Confidence vs Actual Participant Performance
Within each treatment, regress participant self-reported confidence against participant prediction score (controlling for participant’s demographic information and exit survey responses) -> **no statistically significant results**  

In [5]:
# vary treatment number from 0-5 to observe results across treaments
# treatment 0: baseline, 
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers.loc[workers['treatment']==5]

In [6]:
y = treatment['confidence']
X = treatment[['participant_brier_score','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'self_reported_influence', 'self_reported_exp_usefulness', 'self_reported_ra_accuracy', 'self_reported_ra_fairness', 'self_reported_exp_ability', 'accountability', 'ml_fam', 'cj_fam']]

In [7]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [8]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             confidence   R-squared (uncentered):                   0.972
Model:                            OLS   Adj. R-squared (uncentered):              0.964
Method:                 Least Squares   F-statistic:                              119.8
Date:                Mon, 06 Dec 2021   Prob (F-statistic):                    1.47e-47
Time:                        10:53:00   Log-Likelihood:                         -86.774
No. Observations:                  93   AIC:                                      215.5
Df Residuals:                      72   BIC:                                      268.7
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
participant_brier_score                   0.7283      0.639      1.139      0.258      -0.546       2.003
participant_age_zscore                   -0.1240      0.089     -1.388      0.169      -0.302       0.054
participant_degree_zscore                -0.1081      0.071     -1.533      0.130      -0.249       0.032
self_reported_influence                  -0.1136      0.095     -1.194      0.236      -0.303       0.076
self_reported_exp_usefulness              0.1707      0.096      1.785      0.078      -0.020       0.361
self_reported_ra_accuracy                 0.0969      0.120      0.807      0.423      -0.143       0.336
self_reported_ra_fairness                 0.1557      0.106      1.466      0.147      -0.056       0.367
self_reported_exp_ability                 0.2269      0.090      2.521      0.014       0.047       0.406
accountability                           -0.0064      0.088     -0.072      0.942      -0.183       0.170
ml_fam                                    0.1235      0.101      1.225      0.225      -0.077       0.324
cj_fam                                    0.3171      0.115      2.761      0.007       0.088       0.546
participant_gender_Male                   0.0443      0.173      0.257      0.798      -0.300       0.388
participant_ethnicity_Asian              -0.1320      0.329     -0.401      0.689      -0.788       0.524
participant_ethnicity_Caucasian          -0.1464      0.234     -0.626      0.533      -0.613       0.320
participant_ethnicity_Native American     0.6125      0.816      0.751      0.455      -1.013       2.238
participant_ethnicity_Other/unknown       1.0258      0.651      1.576      0.120      -0.272       2.324
participant_ethnicity_Two or more         0.9157      0.825      1.109      0.271      -0.730       2.561
participant_politics_Don't know          -1.0519      0.794     -1.325      0.189      -2.634       0.530
participant_politics_Independent         -0.0334      0.200     -0.167      0.868      -0.432       0.365
participant_politics_Republican          -0.0478      0.244     -0.196      0.845      -0.535       0.440
participant_politics_Something else       0.2478      0.490      0.506      0.614      -0.728       1.224
==============================================================================
Omnibus:                        2.895   Durbin-Watson:                   1.838
Prob(Omnibus):                  0.235   Jarque-Bera (JB):                2.865
Skew:                           0.414   Prob(JB):                        0.239
Kurtosis:                       2.769   Cond. No.                         116.
=============================================

### 2. Self-reported Confidence (relative to other participants) vs Actual Participant Performance
Within each treatment, regress participant self-reported confidence relative to other participants against participant prediction score (controlling for participant’s demographic information and exit survey responses) -> **no statistically significant results**  

In [9]:
# vary treatment number from 0-5 to observe results across treaments
# treatment 0: baseline, 
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers.loc[workers['treatment']==5]

In [10]:
y = treatment['relative_confidence']
X = treatment[['rank','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'self_reported_influence', 'self_reported_exp_usefulness', 'self_reported_ra_accuracy', 'self_reported_ra_fairness', 'self_reported_exp_ability', 'accountability', 'ml_fam', 'cj_fam']]

In [11]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [12]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     relative_confidence   R-squared (uncentered):                   0.979
Model:                             OLS   Adj. R-squared (uncentered):              0.972
Method:                  Least Squares   F-statistic:                              157.1
Date:                 Mon, 06 Dec 2021   Prob (F-statistic):                    1.15e-51
Time:                         10:53:00   Log-Likelihood:                         -70.008
No. Observations:                   93   AIC:                                      182.0
Df Residuals:                       72   BIC:                                      235.2
Df Model:                           21                                                  
Covariance Type:             nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
rank                                      0.0003      0.001      0.417      0.678      -0.001       0.002
participant_age_zscore                    0.0320      0.072      0.442      0.660      -0.112       0.177
participant_degree_zscore                 0.0975      0.059      1.654      0.103      -0.020       0.215
self_reported_influence                   0.0778      0.079      0.986      0.328      -0.080       0.235
self_reported_exp_usefulness              0.1060      0.080      1.321      0.191      -0.054       0.266
self_reported_ra_accuracy                 0.0657      0.100      0.656      0.514      -0.134       0.265
self_reported_ra_fairness                 0.2908      0.085      3.435      0.001       0.122       0.460
self_reported_exp_ability                 0.2581      0.079      3.284      0.002       0.101       0.415
accountability                            0.0322      0.070      0.463      0.645      -0.106       0.171
ml_fam                                    0.0502      0.084      0.600      0.550      -0.116       0.217
cj_fam                                    0.1134      0.098      1.152      0.253      -0.083       0.310
participant_gender_Male                   0.2851      0.144      1.975      0.052      -0.003       0.573
participant_ethnicity_Asian               0.2348      0.267      0.880      0.382      -0.297       0.767
participant_ethnicity_Caucasian           0.0361      0.190      0.190      0.850      -0.343       0.415
participant_ethnicity_Native American     0.6166      0.686      0.899      0.372      -0.751       1.984
participant_ethnicity_Other/unknown      -0.4445      0.556     -0.799      0.427      -1.553       0.664
participant_ethnicity_Two or more        -0.2893      0.693     -0.418      0.678      -1.671       1.092
participant_politics_Don't know          -0.9505      0.661     -1.437      0.155      -2.269       0.368
participant_politics_Independent         -0.0116      0.169     -0.069      0.946      -0.349       0.326
participant_politics_Republican           0.0595      0.212      0.280      0.780      -0.364       0.483
participant_politics_Something else       0.5060      0.400      1.265      0.210      -0.292       1.304
==============================================================================
Omnibus:                        0.184   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.912   Jarque-Bera (JB):                0.271
Skew:                          -0.101   Prob(JB):                        0.873
Kurtosis:                       2.829   Cond. No.                     2.52e+03
==================================

### 3. Assessment of Risk Assessment Model Accuracy
Within each treatment, regress participant perception of risk assessment model accuracy against actual risk assessment model accuracy (prediction score) (controlling for participant’s performance, demographic information, and exit survey responses) -> **positive & statistically significant association in the diverse counterfactual treatment only -> able to evaluate risk assessment model accuracy**

In [13]:
# vary treatment number from 1-5 to observe results across treaments
# treatment 0: baseline, 
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers.loc[workers['treatment']==2]

In [14]:
y = treatment['self_reported_ra_accuracy']
X = treatment[['ra_brier_score','participant_brier_score','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'self_reported_influence', 'self_reported_exp_usefulness', 'confidence', 'relative_confidence', 'self_reported_exp_ability', 'accountability', 'self_reported_ra_fairness', 'ml_fam', 'cj_fam']]

In [15]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [16]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                    OLS Regression Results                                    
==============================================================================================
Dep. Variable:     self_reported_ra_accuracy   R-squared (uncentered):                   0.979
Model:                                   OLS   Adj. R-squared (uncentered):              0.972
Method:                        Least Squares   F-statistic:                              150.7
Date:                       Mon, 06 Dec 2021   Prob (F-statistic):                    9.41e-49
Time:                               10:53:00   Log-Likelihood:                         -62.588
No. Observations:                         89   AIC:                                      167.2
Df Residuals:                             68   BIC:                                      219.4
Df Model:                                 21                                                  
Covariance Type:                   nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
ra_brier_score                            2.8519      1.370      2.081      0.041       0.117       5.586
participant_brier_score                  -2.0707      1.332     -1.555      0.125      -4.728       0.586
participant_age_zscore                    0.0801      0.071      1.131      0.262      -0.061       0.221
participant_degree_zscore                 0.0153      0.076      0.201      0.841      -0.137       0.167
self_reported_influence                   0.3216      0.088      3.657      0.000       0.146       0.497
self_reported_exp_usefulness              0.0158      0.070      0.227      0.821      -0.124       0.155
confidence                                0.2187      0.113      1.932      0.058      -0.007       0.444
relative_confidence                       0.0433      0.123      0.353      0.725      -0.201       0.288
self_reported_exp_ability                 0.0439      0.082      0.537      0.593      -0.119       0.207
accountability                           -0.1797      0.090     -1.985      0.051      -0.360       0.001
self_reported_ra_fairness                 0.4322      0.099      4.358      0.000       0.234       0.630
ml_fam                                   -0.0307      0.107     -0.288      0.775      -0.244       0.182
cj_fam                                   -0.1172      0.119     -0.981      0.330      -0.356       0.121
participant_gender_Male                   0.0189      0.133      0.142      0.888      -0.247       0.284
participant_ethnicity_Asian              -0.8878      0.540     -1.644      0.105      -1.966       0.190
participant_ethnicity_Caucasian          -0.0179      0.231     -0.077      0.939      -0.479       0.444
participant_ethnicity_Latino/hispanic     0.3435      0.358      0.958      0.341      -0.372       1.059
participant_ethnicity_Native American    -0.0302      0.323     -0.093      0.926      -0.675       0.614
participant_politics_Independent         -0.1363      0.237     -0.574      0.568      -0.610       0.337
participant_politics_Republican          -0.0713      0.145     -0.491      0.625      -0.361       0.219
participant_politics_Something else       0.9986      0.482      2.070      0.042       0.036       1.961
==============================================================================
Omnibus:                        2.752   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.253   Jarque-Bera (JB):                2.161
Skew:                          -0.364   Prob(JB):                        0.339
Kurtosis:                       3.230   Cond. N

### 4. Assessment of Risk Assessment Model Fairness
Within each treatment, regress participant perception of risk assessment model fairness against actual risk assessment model fairness (difference in false positive rates for black and white defendants) (controlling for participant's performance, demographic information, exit survey responses, and risk assessment model’s prediction score) -> **negative & statistically significant association in complete feature attribution treatment -> able to dicern how fair the risk assessment model is** 

In [17]:
# vary treatment number from 1-5 to observe results across treaments
# treatment 0: baseline, 
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers.loc[workers['treatment']==4]

# remove instances with more risk assessment false positives for white than black defendants
# to focus on the most salient aspects of bias
treatment = treatment.loc[treatment['false_positive_ra_diff']>=0]

In [18]:
y = treatment['self_reported_ra_fairness']
X = treatment[['false_positive_ra_diff','ra_brier_score','participant_brier_score','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'self_reported_influence', 'self_reported_exp_usefulness', 'confidence', 'relative_confidence', 'self_reported_exp_ability', 'accountability', 'self_reported_ra_accuracy', 'ml_fam', 'cj_fam']]

In [19]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [20]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                    OLS Regression Results                                    
==============================================================================================
Dep. Variable:     self_reported_ra_fairness   R-squared (uncentered):                   0.972
Model:                                   OLS   Adj. R-squared (uncentered):              0.961
Method:                        Least Squares   F-statistic:                              95.00
Date:                       Mon, 06 Dec 2021   Prob (F-statistic):                    7.47e-41
Time:                               10:53:00   Log-Likelihood:                         -70.923
No. Observations:                         87   AIC:                                      187.8
Df Residuals:                             64   BIC:                                      244.6
Df Model:                                 23                                                  
Covariance Type:                   nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
false_positive_ra_diff                   -0.1296      0.056     -2.297      0.025      -0.242      -0.017
ra_brier_score                           -1.5880      1.794     -0.885      0.379      -5.171       1.995
participant_brier_score                   2.0591      1.662      1.239      0.220      -1.261       5.380
participant_age_zscore                   -0.1199      0.080     -1.501      0.138      -0.279       0.040
participant_degree_zscore                 0.0652      0.095      0.690      0.493      -0.124       0.254
self_reported_influence                   0.0481      0.087      0.552      0.583      -0.126       0.222
self_reported_exp_usefulness              0.2386      0.083      2.871      0.006       0.073       0.405
confidence                               -0.0694      0.128     -0.543      0.589      -0.325       0.186
relative_confidence                      -0.0832      0.137     -0.605      0.547      -0.358       0.191
self_reported_exp_ability                 0.1759      0.103      1.703      0.093      -0.030       0.382
accountability                            0.0651      0.081      0.801      0.426      -0.097       0.227
self_reported_ra_accuracy                 0.3404      0.102      3.346      0.001       0.137       0.544
ml_fam                                    0.0394      0.087      0.450      0.654      -0.135       0.214
cj_fam                                    0.1929      0.105      1.843      0.070      -0.016       0.402
participant_gender_Male                   0.3562      0.165      2.157      0.035       0.026       0.686
participant_ethnicity_Asian               0.3536      0.381      0.928      0.357      -0.408       1.115
participant_ethnicity_Caucasian           0.0086      0.293      0.029      0.977      -0.577       0.594
participant_ethnicity_Latino/hispanic    -0.4602      0.411     -1.119      0.268      -1.282       0.362
participant_ethnicity_Native American    -0.8391      0.554     -1.514      0.135      -1.947       0.268
participant_ethnicity_Other/unknown       0.0047      0.748      0.006      0.995      -1.490       1.499
participant_ethnicity_Two or more         1.3416      0.770      1.743      0.086      -0.196       2.880
participant_politics_Independent         -0.1826      0.236     -0.775      0.441      -0.653       0.288
participant_politics_Republican          -0.0392      0.195     -0.201      0.841      -0.429       0.350
==============================================================================
Omnibus:                        0.336   Durbin-Watson:                  

## Calibration
Participants should be able to calibrate their use of the risk assessment model prediction according to the risk assessment model's performance (accuracy and fairness)

### 1. Risk assessment Model's Influence on Participants vs Risk Assessment Model Accuracy
- Within each treatment, regress risk assessment model’s influence on each participant against the risk assessment model’s prediction score for that participant (controlling for participant's performance, demographic information, and exit survey responses) -> **positive and statistically significant associations in all treatments -> able to callibrate properly**

In [21]:
# remove workers who adjusted beyond the risk assessment model
# treatment 0: baseline, 
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
workers_cal = workers.loc[(workers['influence']>=-0.01) & (workers['influence']<=1.01)]

In [22]:
# vary treatment number from 1-5 to observe results across treaments
treatment = workers_cal.loc[workers['treatment']==1]

In [23]:
y = treatment['influence']
X = treatment[['ra_brier_score','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'confidence', 'self_reported_influence', 'self_reported_exp_usefulness', 'self_reported_ra_accuracy', 'self_reported_ra_fairness', 'self_reported_exp_ability', 'accountability', 'relative_confidence', 'ml_fam', 'cj_fam', 'participant_brier_score']]

In [24]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [25]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              influence   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              300.6
Date:                Mon, 06 Dec 2021   Prob (F-statistic):                    2.76e-61
Time:                        10:53:00   Log-Likelihood:                          109.10
No. Observations:                  93   AIC:                                     -174.2
Df Residuals:                      71   BIC:                                     -118.5
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
ra_brier_score                            0.7395      0.270      2.735      0.008       0.200       1.279
participant_age_zscore                    0.0013      0.011      0.114      0.909      -0.021       0.023
participant_degree_zscore                 0.0139      0.013      1.094      0.277      -0.011       0.039
confidence                               -0.0120      0.015     -0.791      0.432      -0.042       0.018
self_reported_influence                  -0.0054      0.012     -0.450      0.654      -0.029       0.018
self_reported_exp_usefulness              0.0211      0.012      1.813      0.074      -0.002       0.044
self_reported_ra_accuracy                -0.0070      0.018     -0.382      0.703      -0.044       0.030
self_reported_ra_fairness                -0.0111      0.014     -0.769      0.444      -0.040       0.018
self_reported_exp_ability                -0.0023      0.014     -0.165      0.869      -0.030       0.025
accountability                            0.0207      0.011      1.912      0.060      -0.001       0.042
relative_confidence                       0.0154      0.017      0.907      0.368      -0.018       0.049
ml_fam                                   -0.0157      0.011     -1.453      0.151      -0.037       0.006
cj_fam                                    0.0043      0.015      0.286      0.776      -0.026       0.035
participant_brier_score                   0.0847      0.262      0.324      0.747      -0.437       0.607
participant_gender_Male                   0.0093      0.022      0.419      0.676      -0.035       0.053
participant_ethnicity_Asian               0.0066      0.035      0.190      0.850      -0.063       0.076
participant_ethnicity_Caucasian           0.0413      0.028      1.472      0.145      -0.015       0.097
participant_ethnicity_Latino/hispanic    -0.0489      0.069     -0.714      0.478      -0.186       0.088
participant_ethnicity_Native American     0.0711      0.080      0.894      0.375      -0.088       0.230
participant_ethnicity_Two or more         0.0004      0.074      0.005      0.996      -0.147       0.148
participant_politics_Independent          0.0032      0.034      0.093      0.926      -0.065       0.071
participant_politics_Republican           0.0230      0.029      0.798      0.427      -0.034       0.080
participant_politics_Something else       0.0066      0.035      0.190      0.850      -0.063       0.076
==============================================================================
Omnibus:                        0.272   Durbin-Watson:                   1.782
Prob(Omnibus):                  0.873   Jarque-Bera (JB):             

### 2. Risk assessment Model's Influence on Participants vs Risk Assessment Model Fairness
- Within each treatment, regress risk assessment model’s influence on each participant against the risk assessment model’s fairness (difference in false positive rates for black and white defendants) for that participant (controlling for participant's performance, demographic information, exit survey responses, and risk assessment model performance) -> **no statistically significant results**

In [26]:
# remove workers who adjusted beyond the risk assessment model
workers_cal = workers.loc[(workers['influence']>=-0.01) & (workers['influence']<=1.01)]

# remove instances with more risk assessment model false positives for white than black defendants
# to focus on the most salient aspects of bias
workers_cal = workers_cal.loc[workers_cal['false_positive_ra_diff']>=0]

In [27]:
# vary treatment number from 1-5 to observe results across treaments
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers_cal.loc[workers['treatment']==5]

In [28]:
y = treatment['influence']
X = treatment[['false_positive_ra_diff','ra_brier_score','participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'confidence', 'self_reported_influence', 'self_reported_exp_usefulness', 'self_reported_ra_accuracy', 'self_reported_ra_fairness', 'self_reported_exp_ability', 'accountability', 'relative_confidence', 'ml_fam', 'cj_fam', 'participant_brier_score']]

In [29]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [30]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              influence   R-squared (uncentered):                   0.992
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              285.4
Date:                Mon, 06 Dec 2021   Prob (F-statistic):                    7.62e-52
Time:                        10:53:00   Log-Likelihood:                          110.42
No. Observations:                  83   AIC:                                     -170.8
Df Residuals:                      58   BIC:                                     -110.4
Df Model:                          25                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
false_positive_ra_diff                   -0.0008      0.006     -0.120      0.905      -0.013       0.012
ra_brier_score                            0.9204      0.190      4.832      0.000       0.539       1.302
participant_age_zscore                    0.0208      0.011      1.951      0.056      -0.001       0.042
participant_degree_zscore                 0.0074      0.009      0.831      0.409      -0.010       0.025
confidence                               -0.0067      0.015     -0.444      0.659      -0.037       0.024
self_reported_influence                  -0.0133      0.012     -1.136      0.261      -0.037       0.010
self_reported_exp_usefulness              0.0109      0.011      0.969      0.337      -0.012       0.034
self_reported_ra_accuracy                -0.0084      0.014     -0.584      0.561      -0.037       0.020
self_reported_ra_fairness                -0.0040      0.013     -0.307      0.760      -0.030       0.022
self_reported_exp_ability                -0.0116      0.014     -0.860      0.393      -0.039       0.015
accountability                           -0.0027      0.012     -0.233      0.816      -0.026       0.021
relative_confidence                      -0.0009      0.020     -0.042      0.967      -0.042       0.040
ml_fam                                    0.0284      0.012      2.369      0.021       0.004       0.052
cj_fam                                -7.354e-05      0.015     -0.005      0.996      -0.030       0.029
participant_brier_score                  -0.0346      0.188     -0.184      0.854      -0.410       0.341
participant_gender_Male                   0.0141      0.021      0.667      0.507      -0.028       0.057
participant_ethnicity_Asian               0.0964      0.038      2.517      0.015       0.020       0.173
participant_ethnicity_Caucasian           0.0219      0.028      0.788      0.434      -0.034       0.078
participant_ethnicity_Native American     0.0695      0.094      0.743      0.460      -0.118       0.257
participant_ethnicity_Other/unknown      -0.0252      0.086     -0.293      0.770      -0.197       0.147
participant_ethnicity_Two or more         0.0624      0.094      0.664      0.509      -0.126       0.251
participant_politics_Don't know          -0.1210      0.090     -1.345      0.184      -0.301       0.059
participant_politics_Independent          0.0605      0.024      2.498      0.015       0.012       0.109
participant_politics_Republican          -0.0252      0.029     -0.864      0.391      -0.083       0.033
participant_politics_Something else       0.0486      0.072      0.674      0.503      -0.096       0.193
================

## Accountability vs Influence
- Do participants calibrate how much they should be held accountable for the decisions they make according to how much influence the algorithm had on them (or how much influence they think it had on them)? And if not, then according to what factors do they calibrate it? 
- The answer to this survey question was used: "If one of the decisions you made goes wrong or is questioned, how much accountability do you think you should face?" Responses were relative to the algorithm developers: (1) none, (2) less than the developers of the algorithm, (3) equal to the developers of the algorithm, (4) more than the developers of the algorithm, (5) I should face accountability, but the developers of the algorithm should not. 

1. **Risk assessment model only treatment:** confidence (+)
2. **Diverse counterfactual treatment:** confidence (+), influence (+), perception of risk assessment model fairness (+)
3. **Selective counterfactual treatment:** N/A
4. **Complete feature attribution treatment:** confidence (+)
5. **Selective feature attribution treatment:** N/A

In [31]:
# remove workers who adjusted beyond the risk assessment model
workers_cal = workers_cal.loc[(workers_cal['influence']>=-0.01) & (workers_cal['influence']<=1.01)]

In [32]:
# vary treatment number from 1-5 to observe results across treaments
# treatment 1: risk assessment model only (unexplained),
# treatment 2: diverse counterfactual,
# treatment 3: selective counterfactual,
# treatment 4: complete feature attribution,
# treatment 5: selective feature attribution,
treatment = workers.loc[(workers['treatment']==2)]

In [33]:
y = treatment['accountability'] 
X = treatment[['influence', 'confidence', 'self_reported_influence', 'participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'self_reported_exp_usefulness', 'self_reported_ra_accuracy', 'self_reported_ra_fairness', 'self_reported_exp_ability', 'relative_confidence', 'ml_fam', 'cj_fam', 'participant_brier_score', 'ra_brier_score']]

In [34]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [35]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         accountability   R-squared (uncentered):                   0.967
Model:                            OLS   Adj. R-squared (uncentered):              0.956
Method:                 Least Squares   F-statistic:                              88.30
Date:                Mon, 06 Dec 2021   Prob (F-statistic):                    5.18e-41
Time:                        10:53:00   Log-Likelihood:                         -79.836
No. Observations:                  89   AIC:                                      203.7
Df Residuals:                      67   BIC:                                      258.4
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
influence                                 2.9456      0.923      3.191      0.002       1.103       4.788
confidence                                0.2784      0.139      2.008      0.049       0.002       0.555
self_reported_influence                   0.1108      0.117      0.948      0.347      -0.122       0.344
participant_age_zscore                   -0.0507      0.088     -0.576      0.566      -0.226       0.125
participant_degree_zscore                 0.2644      0.094      2.798      0.007       0.076       0.453
self_reported_exp_usefulness             -0.0760      0.085     -0.894      0.375      -0.246       0.094
self_reported_ra_accuracy                -0.2429      0.145     -1.670      0.100      -0.533       0.047
self_reported_ra_fairness                 0.4466      0.126      3.534      0.001       0.194       0.699
self_reported_exp_ability                 0.0684      0.100      0.687      0.495      -0.131       0.267
relative_confidence                      -0.0991      0.151     -0.658      0.513      -0.400       0.202
ml_fam                                   -0.0325      0.132     -0.246      0.806      -0.296       0.231
cj_fam                                   -0.0912      0.148     -0.617      0.539      -0.386       0.204
participant_brier_score                   0.1444      1.690      0.085      0.932      -3.229       3.518
ra_brier_score                           -0.2267      1.987     -0.114      0.910      -4.193       3.739
participant_gender_Male                  -0.1432      0.165     -0.867      0.389      -0.473       0.187
participant_ethnicity_Asian              -1.2600      0.660     -1.910      0.060      -2.577       0.057
participant_ethnicity_Caucasian          -0.1590      0.283     -0.563      0.576      -0.723       0.405
participant_ethnicity_Latino/hispanic     0.4933      0.439      1.124      0.265      -0.383       1.369
participant_ethnicity_Native American    -0.1158      0.395     -0.293      0.770      -0.904       0.672
participant_politics_Independent         -0.0747      0.291     -0.257      0.798      -0.655       0.506
participant_politics_Republican           0.0710      0.179      0.396      0.693      -0.287       0.429
participant_politics_Something else       0.4516      0.608      0.743      0.460      -0.762       1.665
==============================================================================
Omnibus:                        0.198   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.906   Jarque-Bera (JB):                0.384
Skew:                          -0.035   Prob(JB):                        0.825
Kurtosis:         